In [1]:
import sqlite3
import pandas as pd
import csv
import argparse

In [2]:
db = sqlite3.connect("Covid.db")
cur = db.cursor()
con = sqlite3.connect("Covid.db")


In [20]:
df = pd.read_csv(r"/Users/jenny.wade@schibsted.com/Documents/GitHub/python_programmering/Inlämning/covid_vaccinations.csv")
df = df.fillna(0)
db = df.to_sql("Covid_vaccinations", con, schema=None, if_exists="replace", index=True, index_label=None, chunksize=None, dtype=None, method=None)


In [21]:
#Skapa dataframes från queries i en linje kod
df = pd.read_sql_query("SELECT * FROM Covid_vaccinations", con)
print(df.head())

   index  Unnamed: 0      country iso_code        date  total_vaccinations  \
0      0           0  Afghanistan      AFG  2021-02-22                 0.0   
1      1           1  Afghanistan      AFG  2021-02-23                 0.0   
2      2           2  Afghanistan      AFG  2021-02-24                 0.0   
3      3           3  Afghanistan      AFG  2021-02-25                 0.0   
4      4           4  Afghanistan      AFG  2021-02-26                 0.0   

   people_vaccinated  people_fully_vaccinated  daily_vaccinations  \
0                0.0                      0.0                 0.0   
1                0.0                      0.0              1367.0   
2                0.0                      0.0              1367.0   
3                0.0                      0.0              1367.0   
4                0.0                      0.0              1367.0   

   total_vaccinations_per_hundred  ...  Covaxin  CanSino  Abdala  \
0                             0.0  ...        0 

In [ ]:
cur.execute("CREATE TABLE Country_Source1 AS SELECT DISTINCT country, source_name, source_website FROM Covid_vaccinations")
con.commit()

In [3]:
cur.execute("CREATE TABLE IF NOT EXISTS Country_Source(country text PRIMARY KEY NOT NULL, source_name text NOT NULL, source_website text NOT NULL)")
cur.execute("INSERT INTO Country_Source SELECT * FROM Country_Source1")

In [4]:

con.commit()

In [3]:
cur.execute("CREATE TABLE IF NOT EXISTS Country_Vaccines(country text PRIMARY KEY, JohnsonAndJohnson integer, OxfordAstraZeneca integer, Sinovac integer \
    SputnikV integer, Moderna integer, SinopharmBeijing integer, Covaxin integer, CanSino integer, Abdala integer, PfizerBioNTech integer, \
    QazVac integer, EpiVacCorona integer, Soberana02 integer, SinopharmHayatVax integer, RBDDimer integer, SinopharmWuhan integer)")
con.commit()

In [4]:
cur.execute("CREATE TABLE Country_Vaccines1 AS SELECT DISTINCT country, JohnsonAndJohnson, OxfordAstraZeneca, Sinovac, \
    SputnikV, Moderna, SinopharmBeijing, Covaxin, CanSino, Abdala, PfizerBioNTech, \
    QazVac, EpiVacCorona, Soberana02, SinopharmHayatVax, RBDDimer, SinopharmWuhan FROM Covid_vaccinations")
con.commit()

In [8]:
cur.execute("CREATE TABLE IF NOT EXISTS Country_Total(total_vaccinations integer PRIMARY KEY, country text , people_vaccinated integer, people_fully_vaccinated integer, \
    total_vaccinations_per_hundred real, people_vaccinated_per_hundred real, people_fully_vaccinated_per_hundred)")
con.commit()

In [13]:
con.commit()
con.close()

In [11]:
from Classes import CleanCSV

    

TypeError: __init__() missing 1 required positional argument: 'database'